In [ ]:
!pip -q install numpy pandas matplotlib scikit-image pillow requests tqdm tensorflow-cpu tensorflow-datasets tensorflow


: 

In [ ]:
import os, shutil, tarfile, zipfile, random
from pathlib import Path
import requests
from tqdm import tqdm

import numpy as np
import pandas as pd
from PIL import Image

import tensorflow as tf
import tensorflow_datasets as tfds

# Rutas del proyecto
ROOT = Path(".")
RAW_DIR = ROOT / "data" / "raw"
IMG_DIR = ROOT / "data" / "images"
RAW_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)

def download_file(url: str, dst: Path, chunk_size=1<<20):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists() and dst.stat().st_size > 0:
        print(f"[OK] Ya existe: {dst.name}")
        return dst

    print(f"[DL] {url}")
    r = requests.get(url, stream=True, timeout=60)
    r.raise_for_status()
    total = int(r.headers.get("content-length", 0))
    with open(dst, "wb") as f, tqdm(total=total, unit="B", unit_scale=True) as pbar:
        for chunk in r.iter_content(chunk_size=chunk_size):
            if chunk:
                f.write(chunk)
                pbar.update(len(chunk))
    return dst

def extract_zip(zip_path: Path, out_dir: Path):
    print(f"[EXTRACT ZIP] {zip_path.name} -> {out_dir}")
    out_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(out_dir)

def extract_tar_xz(tar_path: Path, out_dir: Path):
    print(f"[EXTRACT TAR.XZ] {tar_path.name} -> {out_dir}")
    out_dir.mkdir(parents=True, exist_ok=True)
    with tarfile.open(tar_path, mode="r:xz") as t:
        t.extractall(out_dir)

def list_images(root: Path, exts=(".png",".jpg",".jpeg",".tif",".tiff",".bmp",".webp")):
    files = []
    for p in root.rglob("*"):
        if p.is_file() and p.suffix.lower() in exts:
            files.append(p)
    return sorted(files)

def save_as_png(src: Path, dst: Path, max_side=1200):
    dst.parent.mkdir(parents=True, exist_ok=True)
    img = Image.open(src).convert("RGB")
    # reescalado moderado para no meter imágenes gigantes
    w, h = img.size
    s = max(w, h)
    if s > max_side:
        scale = max_side / s
        img = img.resize((int(w*scale), int(h*scale)), Image.Resampling.LANCZOS)
    img.save(dst, format="PNG", optimize=True)
    return dst


In [ ]:
# URL de descarga (LHNCBC). Si falla, entra en la página y copia el enlace de "Download the dataset".
INDIANA_ZIP_URL = "https://data.lhncbc.nlm.nih.gov/public/chest-xray/indiana.zip"

med_dir = RAW_DIR / "medical_indiana"
zip_path = RAW_DIR / "indiana.zip"

download_file(INDIANA_ZIP_URL, zip_path)
extract_zip(zip_path, med_dir)

med_imgs = list_images(med_dir)
print("Imágenes médicas encontradas:", len(med_imgs))
med_imgs[:5]


[DL] https://data.lhncbc.nlm.nih.gov/public/chest-xray/indiana.zip


100%|██████████| 239M/239M [00:04<00:00, 54.8MB/s]


[EXTRACT ZIP] indiana.zip -> data/raw/medical_indiana
Imágenes médicas encontradas: 440


[PosixPath('data/raw/medical_indiana/__MACOSX/indiana/CXR_png/._1036_IM-0029-1001.png'),
 PosixPath('data/raw/medical_indiana/__MACOSX/indiana/CXR_png/._1039_IM-0030-1002.png'),
 PosixPath('data/raw/medical_indiana/__MACOSX/indiana/CXR_png/._1042_IM-0034-1001.png'),
 PosixPath('data/raw/medical_indiana/__MACOSX/indiana/CXR_png/._1045_IM-0036-1001.png'),
 PosixPath('data/raw/medical_indiana/__MACOSX/indiana/CXR_png/._1051_IM-0039-5001.png')]

In [ ]:
from pathlib import Path
import random
import numpy as np
from tqdm import tqdm
from PIL import Image

from skimage.feature import canny
from skimage import exposure

# 1) Re-filtra imágenes médicas SOLO a formatos típicamente válidos
med_imgs = [p for p in med_dir.rglob("*") if p.suffix.lower() in [".png", ".jpg", ".jpeg"]]
print("Imágenes médicas (png/jpg) encontradas:", len(med_imgs))
print("Ejemplo:", med_imgs[:5])

def edge_density_gray_safe(path: Path):
    """
    Lee con PIL (más tolerante), pasa a gris, normaliza y calcula densidad de bordes.
    Si falla, devuelve np.nan para poder descartarlo.
    """
    try:
        img = Image.open(path).convert("L")   # gris
        x = np.asarray(img, dtype=np.float32) / 255.0
        x = exposure.rescale_intensity(x, in_range="image", out_range=(0, 1))
        e = canny(x, sigma=1.2)
        return float(e.mean())
    except Exception as ex:
        # opcional: imprime qué fichero falló
        # print(f"[SKIP] {path.name}: {ex}")
        return np.nan

# 2) Calculamos densidad de bordes sobre un subconjunto
sample = med_imgs if len(med_imgs) <= 200 else random.sample(med_imgs, 200)

scores = []
for p in tqdm(sample):
    s = edge_density_gray_safe(p)
    if not np.isnan(s):
        scores.append((p, s))

print("Imágenes válidas para scoring:", len(scores), "de", len(sample))

# 3) Selección por percentiles (variedad)
scores.sort(key=lambda t: t[1])

p20 = scores[int(0.2 * len(scores))][0]
p50 = scores[int(0.5 * len(scores))][0]
p80 = scores[int(0.8 * len(scores))][0]

medical_selected = [p20, p50, p80]
medical_selected



Imágenes médicas (png/jpg) encontradas: 110
Ejemplo: [PosixPath('data/raw/medical_indiana/indiana/CXR_png/960_IM-2451-4004.png'), PosixPath('data/raw/medical_indiana/indiana/CXR_png/8_IM-2333-1001.png'), PosixPath('data/raw/medical_indiana/indiana/CXR_png/965_IM-2455-1001.png'), PosixPath('data/raw/medical_indiana/indiana/CXR_png/943_IM-2439-5005.png'), PosixPath('data/raw/medical_indiana/indiana/CXR_png/11_IM-0067-1001.png')]


100%|██████████| 110/110 [01:00<00:00,  1.83it/s]

Imágenes válidas para scoring: 55 de 110


[PosixPath('data/raw/medical_indiana/indiana/CXR_png/999_IM-2480-1001.png'),
 PosixPath('data/raw/medical_indiana/indiana/CXR_png/958_IM-2449-1001.png'),
 PosixPath('data/raw/medical_indiana/indiana/CXR_png/986_IM-2473-2002.png')]

In [ ]:
MVTEC_URLS = {
    "grid":   "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f283/download/420937487-1629959044/grid.tar.xz",
    "leather":"https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f283/download/420937607-1629959262/leather.tar.xz",
    "bottle": "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f283/download/420937370-1629958698/bottle.tar.xz",
}

mvtec_root = RAW_DIR / "mvtec_ad"
mvtec_root.mkdir(parents=True, exist_ok=True)

for cat, url in MVTEC_URLS.items():
    tar_path = RAW_DIR / f"mvtec_{cat}.tar.xz"
    download_file(url, tar_path)
    extract_tar_xz(tar_path, mvtec_root)

print("OK. Carpetas extraídas:", [p.name for p in mvtec_root.iterdir() if p.is_dir()][:10])


[DL] https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f283/download/420937487-1629959044/grid.tar.xz


100%|██████████| 161M/161M [00:09<00:00, 16.1MB/s]
/tmp/ipython-input-2146782007.py:47: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  t.extractall(out_dir)


[EXTRACT TAR.XZ] mvtec_grid.tar.xz -> data/raw/mvtec_ad
[DL] https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f283/download/420937607-1629959262/leather.tar.xz


100%|██████████| 525M/525M [00:44<00:00, 11.9MB/s]


[EXTRACT TAR.XZ] mvtec_leather.tar.xz -> data/raw/mvtec_ad
[DL] https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f283/download/420937370-1629958698/bottle.tar.xz


100%|██████████| 156M/156M [00:09<00:00, 16.3MB/s]


[EXTRACT TAR.XZ] mvtec_bottle.tar.xz -> data/raw/mvtec_ad
OK. Carpetas extraídas: ['grid', 'bottle', 'leather']


In [ ]:
def pick_mvtec_image(cat_dir: Path):
    # Preferimos una defectuosa (test/* excepto good), si existe; si no, usamos good
    test_dir = cat_dir / "test"
    if test_dir.exists():
        # defectuosas
        defect = []
        for sub in test_dir.iterdir():
            if sub.is_dir() and sub.name.lower() != "good":
                defect += list_images(sub)
        if defect:
            return defect[0]  # primera defectuosa
        good = list_images(test_dir / "good")
        if good:
            return good[0]
    # fallback a train
    train_dir = cat_dir / "train"
    if train_dir.exists():
        good = list_images(train_dir / "good")
        if good:
            return good[0]
    # último recurso
    imgs = list_images(cat_dir)
    return imgs[0] if imgs else None

industrial_selected = []
for cat in ["grid", "leather", "bottle"]:
    cat_dir = mvtec_root / cat
    p = pick_mvtec_image(cat_dir)
    industrial_selected.append(p)

industrial_selected


[PosixPath('data/raw/mvtec_ad/grid/test/metal_contamination/000.png'),
 PosixPath('data/raw/mvtec_ad/leather/test/color/000.png'),
 PosixPath('data/raw/mvtec_ad/bottle/test/contamination/000.png')]

In [ ]:
builder = tfds.builder("eurosat/rgb")
builder.download_and_prepare()

info = builder.info
label_names = info.features["label"].names
print("Labels:", label_names)

target_labels = ["Residential", "River", "Forest"]
target_ids = [label_names.index(x) for x in target_labels]
target_ids


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/eurosat/rgb/incomplete.8C53OG_2.0.0/eurosat-train.tfrecord*...:   0%|     …

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.
Labels: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


[7, 8, 1]

In [ ]:
ds = builder.as_dataset(split="train", shuffle_files=True)
ds_np = tfds.as_numpy(ds)

picked = {lab: None for lab in target_labels}

for ex in ds_np:
    lab_id = int(ex["label"])
    if lab_id in target_ids:
        lab_name = label_names[lab_id]
        if picked[lab_name] is None:
            picked[lab_name] = ex["image"]
        if all(v is not None for v in picked.values()):
            break

sat_selected_arrays = [(k, v) for k, v in picked.items()]
[(k, None if v is None else v.shape) for k, v in sat_selected_arrays]


[('Residential', (64, 64, 3)), ('River', (64, 64, 3)), ('Forest', (64, 64, 3))]

In [ ]:
meta_rows = []
out_paths = []

# 3 médicas
for i, src in enumerate(medical_selected, start=1):
    img_id = f"I{i:02d}"
    dst = IMG_DIR / f"{img_id}.png"
    save_as_png(src, dst)
    meta_rows.append({
        "image_id": img_id,
        "source": "LHNCBC subset (Open-i Indiana CXR)",
        "domain": "medica",
        "label_or_category": "CXR",
        "license_note": "Ver condiciones del recurso (uso académico/no comercial recomendado)",
        "original_path": str(src),
        "saved_path": str(dst),
    })
    out_paths.append(dst)

# 3 industriales
base = 4
for j, src in enumerate(industrial_selected, start=0):
    img_id = f"I{base+j:02d}"
    dst = IMG_DIR / f"{img_id}.png"
    save_as_png(src, dst)
    meta_rows.append({
        "image_id": img_id,
        "source": "MVTec AD (category download)",
        "domain": "industrial",
        "label_or_category": src.parent.name if src else "NA",
        "license_note": "CC BY-NC-SA 4.0 (no comercial)",
        "original_path": str(src),
        "saved_path": str(dst),
    })
    out_paths.append(dst)

# 3 satélite
base = 7
for j, (lab_name, arr) in enumerate(sat_selected_arrays, start=0):
    img_id = f"I{base+j:02d}"
    dst = IMG_DIR / f"{img_id}.png"
    # arr es uint8 RGB
    Image.fromarray(arr).save(dst, format="PNG", optimize=True)
    meta_rows.append({
        "image_id": img_id,
        "source": "EuroSAT (TFDS eurosat/rgb)",
        "domain": "satelite",
        "label_or_category": lab_name,
        "license_note": "Según licencia/dataset card de EuroSAT",
        "original_path": "TFDS internal",
        "saved_path": str(dst),
    })
    out_paths.append(dst)

meta = pd.DataFrame(meta_rows)
meta_path = ROOT / "meta.csv"
meta.to_csv(meta_path, index=False)

print("Guardadas:", len(out_paths), "imágenes en", IMG_DIR)
display(meta)


Guardadas: 9 imágenes en data/images


,image_id,source,domain,label_or_category,license_note,original_path,saved_path
0,I01,LHNCBC subset (Open-i Indiana CXR),medica,CXR,Ver condiciones del recurso (uso académico/no ...,data/raw/medical_indiana/indiana/CXR_png/999_I...,data/images/I01.png
1,I02,LHNCBC subset (Open-i Indiana CXR),medica,CXR,Ver condiciones del recurso (uso académico/no ...,data/raw/medical_indiana/indiana/CXR_png/958_I...,data/images/I02.png
2,I03,LHNCBC subset (Open-i Indiana CXR),medica,CXR,Ver condiciones del recurso (uso académico/no ...,data/raw/medical_indiana/indiana/CXR_png/986_I...,data/images/I03.png
3,I04,MVTec AD (category download),industrial,metal_contamination,CC BY-NC-SA 4.0 (no comercial),data/raw/mvtec_ad/grid/test/metal_contaminatio...,data/images/I04.png
4,I05,MVTec AD (category download),industrial,color,CC BY-NC-SA 4.0 (no comercial),data/raw/mvtec_ad/leather/test/color/000.png,data/images/I05.png
5,I06,MVTec AD (category download),industrial,contamination,CC BY-NC-SA 4.0 (no comercial),data/raw/mvtec_ad/bottle/test/contamination/00...,data/images/I06.png
6,I07,EuroSAT (TFDS eurosat/rgb),satelite,Residential,Según licencia/dataset card de EuroSAT,TFDS internal,data/images/I07.png
7,I08,EuroSAT (TFDS eurosat/rgb),satelite,River,Según licencia/dataset card de EuroSAT,TFDS internal,data/images/I08.png
8,I09,EuroSAT (TFDS eurosat/rgb),satelite,Forest,Según licencia/dataset card de EuroSAT,TFDS internal,data/images/I09.png


In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import io, color, img_as_float
from skimage.util import img_as_ubyte
from skimage import exposure
from skimage.filters import gaussian, median, sobel, laplace, unsharp_mask, threshold_otsu
from skimage.feature import canny
from skimage.morphology import disk, erosion, dilation, opening, closing, white_tophat, black_tophat
from skimage.metrics import structural_similarity as ssim
from skimage.restoration import estimate_sigma
from skimage.measure import label, regionprops_table, shannon_entropy

DATA_DIR = Path("data/images")
OUT_DIR = Path("outputs")
(OUT_DIR / "montages").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "hists").mkdir(parents=True, exist_ok=True)

def load_image(path: Path):
    img = io.imread(path)
    img_f = img_as_float(img)
    if img_f.ndim == 3:
        # Convertimos a gris para análisis comparable (en el informe puedes justificarlo)
        img_g = color.rgb2gray(img_f)
    else:
        img_g = img_f
    # Normalizamos a [0,1]
    img_g = exposure.rescale_intensity(img_g, in_range="image", out_range=(0, 1))
    return img_f, img_g

def rms_contrast(x):
    return float(np.std(x))

def sharpness_lapvar(x):
    # Varianza del Laplaciano como proxy de nitidez (mayor -> más detalle/edge content)
    L = laplace(x)
    return float(np.var(L))

def edge_density(x, sigma=1.2):
    e = canny(x, sigma=sigma)
    return float(e.mean())

def histogram(x, bins=256):
    h, _ = np.histogram(x.ravel(), bins=bins, range=(0,1), density=True)
    return h

def hist_intersection(h1, h2):
    return float(np.minimum(h1, h2).sum())

def binarize_otsu(x):
    t = threshold_otsu(x)
    return (x >= t).astype(np.uint8)


In [ ]:
image_paths = sorted([p for p in DATA_DIR.glob("*") if p.suffix.lower() in [".png",".jpg",".jpeg",".tif",".tiff",".bmp",".webp"]])
print("N imágenes:", len(image_paths))
for p in image_paths:
    print("-", p.name)

# Tabla de metadatos editable
meta = pd.DataFrame({
    "image_id": [p.stem for p in image_paths],
    "filename": [p.name for p in image_paths],
    "contexto": ["NA"] * len(image_paths),     # ej.: "medica", "industrial", "satelite", "microscopia"
    "nota": [""] * len(image_paths)            # breve: qué se ve, objetivo, etc.
})
meta


N imágenes: 9
- I01.png
- I02.png
- I03.png
- I04.png
- I05.png
- I06.png
- I07.png
- I08.png
- I09.png


,image_id,filename,contexto,nota
0,I01,I01.png,NA,
1,I02,I02.png,NA,
2,I03,I03.png,NA,
3,I04,I04.png,NA,
4,I05,I05.png,NA,
5,I06,I06.png,NA,
6,I07,I07.png,NA,
7,I08,I08.png,NA,
8,I09,I09.png,NA,


In [ ]:
from skimage.filters import gaussian
from skimage.filters.rank import mean as rank_mean
from skimage.morphology import disk
from skimage import img_as_ubyte
from skimage.restoration import denoise_bilateral, denoise_nl_means, estimate_sigma
from skimage.exposure import equalize_adapthist

# Activa/desactiva filtros más lentos
USE_SLOW_FILTERS = True  # True -> bilateral + NLM

def spatial_filters(x):
    out = {
        # Suavizado clásico
        "gaussian_s1": gaussian(x, 1.0, preserve_range=True),
        "gaussian_s2": gaussian(x, 2.0, preserve_range=True),
        "median_r2": median(x, footprint=disk(2)),
        "median_r4": median(x, footprint=disk(4)),
    }

    # Mean/box filter (baseline sencillo)
    x8 = img_as_ubyte(np.clip(x, 0, 1))
    out["mean_r3"] = rank_mean(x8, footprint=disk(3)).astype(np.float32) / 255.0

    # Contraste (muy útil en CXR / satélite)
    out["clahe"] = equalize_adapthist(np.clip(x, 0, 1), clip_limit=0.01)

    # Realce / bordes
    out["unsharp_r1_a1"] = exposure.rescale_intensity(
        x + (x - gaussian(x, 1.0))*1.0, out_range=(0,1)
    )
    out["unsharp_r2_a15"] = exposure.rescale_intensity(
    x + (x - gaussian(x, 2.0))*1.5, out_range=(0,1)
    )

    out["sobel"] = exposure.rescale_intensity(sobel(x), out_range=(0,1))
    out["laplace"] = exposure.rescale_intensity(np.abs(laplace(x)), out_range=(0,1))
    out["canny_s12"] = canny(x, 1.2).astype(np.float32)

    # Filtros más caros (opcionales)
    if USE_SLOW_FILTERS:
        out["bilateral"] = denoise_bilateral(x, sigma_color=0.08, sigma_spatial=3, channel_axis=None)
        sig = np.mean(estimate_sigma(x, channel_axis=None))
        out["nlm_fast"] = denoise_nl_means(
            x, h=1.1*sig, fast_mode=True, patch_size=5, patch_distance=6, channel_axis=None
        )

    return out


def morph_filters(x, r):
    se = disk(r)
    er = erosion(x, se)
    di = dilation(x, se)
    grad = np.clip(di - er, 0, 1)  # gradiente morfológico (borde)

    return {
        f"opening_r{r}": opening(x, se),
        f"closing_r{r}": closing(x, se),
        f"tophat_r{r}": white_tophat(x, se),
        f"blackhat_r{r}": black_tophat(x, se),
        f"erosion_r{r}": er,
        f"dilation_r{r}": di,
        f"morphgrad_r{r}": grad,
    }


def pipelines(x):
    den = median(x, footprint=disk(3))
    p1 = exposure.rescale_intensity(den + (den - gaussian(den, 1.5))*1.2, out_range=(0,1))

    # morfología como post-proceso “limpiador”
    p2 = opening(closing(x, disk(3)), disk(3))

    # contraste + bordes (útil en CXR/satélite, con cuidado)
    p3 = equalize_adapthist(np.clip(gaussian(x, 1.0), 0, 1), clip_limit=0.01)

    return {
        "P1_median3_then_unsharp": p1,
        "P2_close3_then_open3": p2,
        "P3_gauss1_then_CLAHE": p3,
    }



In [ ]:
def compute_metrics(x0, x1):
    # x0 = original (gris), x1 = filtrada (gris)
    h0 = histogram(x0)
    h1 = histogram(x1)

    return {
        "ssim_vs_orig": float(ssim(x0, x1, data_range=1.0)),
        "sigma_est": float(estimate_sigma(x1, channel_axis=None, average_sigmas=True)),
        "rms_contrast": rms_contrast(x1),
        "sharp_lapvar": sharpness_lapvar(x1),
        "edge_density": edge_density(x1, sigma=1.2),
        "entropy": float(shannon_entropy(x1)),
        "hist_intersection": hist_intersection(h0, h1),
    }

def compute_morph_stats(x_gray):
    # binarizamos y medimos componentes (útil para ver efectos de apertura/cierre)
    bw = binarize_otsu(x_gray)
    lab = label(bw)
    props = regionprops_table(lab, properties=("area", "perimeter"))
    if len(props["area"]) == 0:
        return {"n_comp": 0, "area_mean": 0.0, "area_sum": 0.0, "perim_mean": 0.0}
    areas = np.array(props["area"], dtype=float)
    perim = np.array(props["perimeter"], dtype=float)
    return {
        "n_comp": int(len(areas)),
        "area_mean": float(areas.mean()),
        "area_sum": float(areas.sum()),
        "perim_mean": float(perim.mean()),
    }


In [ ]:
records = []

for p in image_paths:
    img_color, img = load_image(p)
    sid = p.stem

    # Original metrics baseline
    base_m = compute_metrics(img, img)
    base_s = compute_morph_stats(img)

    # Spatial
    S = spatial_filters(img)
    for name, y in S.items():
        m = compute_metrics(img, y)
        s = compute_morph_stats(y)
        records.append({
            "image_id": sid, "tipo": "espacial", "filtro": name,
            **m, **s
        })

    # Morph (probamos dos radios)
    for r in [2, 4]:
        M = morph_filters(img, r)
        for name, y in M.items():
            m = compute_metrics(img, y)
            s = compute_morph_stats(y)
            records.append({
                "image_id": sid, "tipo": "morfologico", "filtro": name,
                **m, **s
            })

    # Pipelines
    P = pipelines(img)
    for name, y in P.items():
        m = compute_metrics(img, y)
        s = compute_morph_stats(y)
        records.append({
            "image_id": sid, "tipo": "pipeline", "filtro": name,
            **m, **s
        })

    # Montaje (original + 6 resultados representativos)
    picks = [
        ("original", img),
        ("gaussian_s2", S["gaussian_s2"]),
        ("median_r4", S["median_r4"]),
        ("unsharp_r2_a15", S["unsharp_r2_a15"]),
        ("sobel", S["sobel"]),
        ("closing_r4", morph_filters(img, 4)["closing_r4"]),
        ("P1_median3_then_unsharp", P["P1_median3_then_unsharp"]),
    ]

    fig, axes = plt.subplots(2, 4, figsize=(14, 7))
    axes = axes.ravel()
    for ax, (title, im) in zip(axes, picks):
        ax.imshow(im, cmap="gray", vmin=0, vmax=1)
        ax.set_title(title, fontsize=10)
        ax.axis("off")
    for ax in axes[len(picks):]:
        ax.axis("off")
    fig.tight_layout()
    out_path = OUT_DIR / "montages" / f"{sid}_montage.png"
    fig.savefig(out_path, dpi=200)
    plt.close(fig)

    # Histogramas (original vs 2 filtros)
    fig = plt.figure(figsize=(8,5))
    plt.plot(histogram(img), label="orig")
    plt.plot(histogram(S["gaussian_s2"]), label="gauss_s2")
    plt.plot(histogram(S["unsharp_r2_a15"]), label="unsharp")
    plt.legend()
    plt.title(f"Histogramas: {sid}")
    plt.tight_layout()
    out_hist = OUT_DIR / "hists" / f"{sid}_hists.png"
    fig.savefig(out_hist, dpi=200)
    plt.close(fig)

df = pd.DataFrame.from_records(records)
df.to_csv(OUT_DIR / "metrics.csv", index=False)
df.head()


,image_id,tipo,filtro,ssim_vs_orig,sigma_est,rms_contrast,sharp_lapvar,edge_density,entropy,hist_intersection,n_comp,area_mean,area_sum,perim_mean
0,I01,espacial,gaussian_s1,0.926935,0.000644,0.197648,0.000108,0.031061,20.171333,250.669268,139,4697.395683,652938.0,155.880015
1,I01,espacial,gaussian_s2,0.854761,0.000060,0.196415,0.000014,0.019145,20.171333,249.007696,35,18630.828571,652079.0,497.498267
2,I01,espacial,median_r2,0.908172,0.002817,0.198099,0.000515,0.033670,7.596199,253.596965,146,4473.335616,653107.0,148.736961
3,I01,espacial,median_r4,0.851906,0.001740,0.197219,0.000226,0.025669,7.585140,252.103415,45,14657.888889,659605.0,396.070183
4,I01,espacial,mean_r3,0.861410,0.001356,0.196857,0.000064,0.022182,7.597020,248.882602,60,10829.383333,649763.0,310.474399


In [ ]:
# Criterio simple: maximizar SSIM y minimizar sigma_est (normalizado)
def rank_score(sub):
    ssim_n = (sub["ssim_vs_orig"] - sub["ssim_vs_orig"].min()) / (sub["ssim_vs_orig"].max() - sub["ssim_vs_orig"].min() + 1e-9)
    sig_n  = (sub["sigma_est"] - sub["sigma_est"].min()) / (sub["sigma_est"].max() - sub["sigma_est"].min() + 1e-9)
    return ssim_n - 0.6*sig_n

top = []
for img_id, sub in df[df["tipo"].isin(["espacial","pipeline"])].groupby("image_id"):
    sub = sub.copy()
    sub["score"] = rank_score(sub)
    top.append(sub.sort_values("score", ascending=False).head(5))
top_df = pd.concat(top).sort_values(["image_id","score"], ascending=[True,False])
top_df[["image_id","tipo","filtro","score","ssim_vs_orig","sigma_est","rms_contrast","sharp_lapvar","edge_density"]]


In [ ]:
from pathlib import Path
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

from skimage import io, color, exposure
from skimage.feature import canny

# -----------------------
# Config rutas de tu proyecto
# -----------------------
DATA_DIR = Path("data")
RAW_DIR  = DATA_DIR / "raw"
OUT_DIR  = DATA_DIR / "images"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------
# Lectura segura (evita files raros / no imagen)
# -----------------------
def safe_imread(path: Path):
    try:
        return io.imread(path)
    except Exception:
        return None

def edge_density_gray(path: Path):
    x = safe_imread(path)
    if x is None:
        return None
    if x.ndim == 3:
        x = color.rgb2gray(x)
    x = exposure.rescale_intensity(x, in_range="image", out_range=(0,1))
    e = canny(x, sigma=1.2)
    return float(e.mean())

# =========================================================
# A) MÉDICAS (Indiana CXR) -> 3 imágenes por percentiles
# =========================================================
# IMPORTANTE: usar SOLO CXR_png
INDIANA_CXR = RAW_DIR / "medical_indiana" / "indiana" / "CXR_png"
med_imgs = sorted(INDIANA_CXR.glob("*.png"))

if len(med_imgs) == 0:
    raise FileNotFoundError(f"No encuentro PNGs en: {INDIANA_CXR}")

sample = med_imgs if len(med_imgs) <= 200 else random.sample(med_imgs, 200)
scores = []
for p in tqdm(sample, desc="Scoring Indiana CXR"):
    s = edge_density_gray(p)
    if s is not None:
        scores.append((p, s))

scores.sort(key=lambda t: t[1])
p20 = scores[int(0.2*len(scores))][0]
p50 = scores[int(0.5*len(scores))][0]
p80 = scores[int(0.8*len(scores))][0]
medical_selected = [p20, p50, p80]

# =========================================================
# B) INDUSTRIAL (MVTec AD) -> 3 categorías (ejemplos)
# =========================================================
MVTEC = RAW_DIR / "mvtec_ad"
industrial_selected = [
    MVTEC / "grid"   / "test" / "bent"         / "000.png",
    MVTEC / "leather"/ "test" / "fold"         / "000.png",
    MVTEC / "bottle" / "test" / "broken_large" / "000.png",
]
for p in industrial_selected:
    if not p.exists():
        raise FileNotFoundError(f"Falta imagen MVTec: {p}")

# =========================================================
# C) SATÉLITE (EuroSAT via TFDS) -> Residential, River, Forest
# =========================================================
import tensorflow_datasets as tfds

builder = tfds.builder("eurosat/rgb")
builder.download_and_prepare()  # en colab suele ir bien (si ya estaba, no re-descarga)
label_names = builder.info.features["label"].names

ds = tfds.load("eurosat/rgb", split="train", as_supervised=True)

targets = {"Residential": None, "River": None, "Forest": None}
for img, lab in tfds.as_numpy(ds):
    name = label_names[int(lab)]
    if name in targets and targets[name] is None:
        targets[name] = img
    if all(v is not None for v in targets.values()):
        break

if any(v is None for v in targets.values()):
    raise RuntimeError("No he podido recoger (Residential, River, Forest) de EuroSAT.")

satellite_selected = [
    ("Residential", targets["Residential"]),
    ("River", targets["River"]),
    ("Forest", targets["Forest"]),
]

# =========================================================
# Guardado final I01..I09 en data/images + meta
# =========================================================
meta_rows = []
img_id = 1

def save_png(arr, out_path: Path):
    # io.imsave soporta uint8 bien; aseguramos 0..255
    arr = np.asarray(arr)
    if arr.dtype != np.uint8:
        arr = np.clip(arr, 0, 255).astype(np.uint8)
    io.imsave(str(out_path), arr)

# Médicas
for p in medical_selected:
    out = OUT_DIR / f"I{img_id:02d}.png"
    x = io.imread(p)
    save_png(x, out)
    meta_rows.append({
        "image_id": f"I{img_id:02d}",
        "source": "LHNCBC subset (Open-i Indiana CXR)",
        "domain": "medica",
        "label_or_category": "CXR",
        "license_note": "Ver condiciones del recurso (uso académico/no comercial)",
        "original_path": str(p),
        "saved_path": str(out),
    })
    img_id += 1

# Industriales
cats = ["bent", "fold", "broken_large"]
for p, cat in zip(industrial_selected, cats):
    out = OUT_DIR / f"I{img_id:02d}.png"
    x = io.imread(p)
    save_png(x, out)
    meta_rows.append({
        "image_id": f"I{img_id:02d}",
        "source": "MVTec AD (category download)",
        "domain": "industrial",
        "label_or_category": cat,
        "license_note": "CC BY-NC-SA 4.0 (no comercial)",
        "original_path": str(p),
        "saved_path": str(out),
    })
    img_id += 1

# Satélite
for (cls, arr) in satellite_selected:
    out = OUT_DIR / f"I{img_id:02d}.png"
    save_png(arr, out)
    meta_rows.append({
        "image_id": f"I{img_id:02d}",
        "source": "EuroSAT (TFDS eurosat/rgb)",
        "domain": "satelite",
        "label_or_category": cls,
        "license_note": "Según licencia/dataset card de EuroSAT",
        "original_path": "TFDS internal",
        "saved_path": str(out),
    })
    img_id += 1

meta = pd.DataFrame(meta_rows)
meta


Scoring Indiana CXR: 100%|██████████| 55/55 [01:17<00:00,  1.42s/it]
/usr/local/lib/python3.12/dist-packages/skimage/_shared/utils.py:328: UserWarning: data/images/I09.png is a low contrast image
  return func(*args, **kwargs)


,image_id,source,domain,label_or_category,license_note,original_path,saved_path
0,I01,LHNCBC subset (Open-i Indiana CXR),medica,CXR,Ver condiciones del recurso (uso académico/no ...,data/raw/medical_indiana/indiana/CXR_png/999_I...,data/images/I01.png
1,I02,LHNCBC subset (Open-i Indiana CXR),medica,CXR,Ver condiciones del recurso (uso académico/no ...,data/raw/medical_indiana/indiana/CXR_png/958_I...,data/images/I02.png
2,I03,LHNCBC subset (Open-i Indiana CXR),medica,CXR,Ver condiciones del recurso (uso académico/no ...,data/raw/medical_indiana/indiana/CXR_png/986_I...,data/images/I03.png
3,I04,MVTec AD (category download),industrial,bent,CC BY-NC-SA 4.0 (no comercial),data/raw/mvtec_ad/grid/test/bent/000.png,data/images/I04.png
4,I05,MVTec AD (category download),industrial,fold,CC BY-NC-SA 4.0 (no comercial),data/raw/mvtec_ad/leather/test/fold/000.png,data/images/I05.png
5,I06,MVTec AD (category download),industrial,broken_large,CC BY-NC-SA 4.0 (no comercial),data/raw/mvtec_ad/bottle/test/broken_large/000...,data/images/I06.png
6,I07,EuroSAT (TFDS eurosat/rgb),satelite,Residential,Según licencia/dataset card de EuroSAT,TFDS internal,data/images/I07.png
7,I08,EuroSAT (TFDS eurosat/rgb),satelite,River,Según licencia/dataset card de EuroSAT,TFDS internal,data/images/I08.png
8,I09,EuroSAT (TFDS eurosat/rgb),satelite,Forest,Según licencia/dataset card de EuroSAT,TFDS internal,data/images/I09.png


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from skimage import color, exposure
from skimage.filters import gaussian, median, sobel, laplace, threshold_otsu
from skimage.feature import canny
from skimage.morphology import disk, erosion, dilation, opening, closing, white_tophat, black_tophat
from skimage.metrics import structural_similarity as ssim
from skimage.measure import label, regionprops_table, shannon_entropy
from skimage.transform import resize

# -----------------------
# Paths
# -----------------------
IMG_DIR = Path("data/images")
RES_DIR = Path("results")
FIG_DIR = RES_DIR / "figures"
TAB_DIR = RES_DIR / "tables"
FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)

img_paths = [IMG_DIR / f"I{i:02d}.png" for i in range(1, 10)]
assert all(p.exists() for p in img_paths), "Faltan I01..I09 en data/images"

# -----------------------
# Carga + normalización + downscale (para ir rápido)
# -----------------------
def load_gray(path: Path, max_side=512):
    arr = np.asarray(Image.open(path))
    if arr.ndim == 3:
        x = color.rgb2gray(arr.astype(np.float32) / 255.0)
    else:
        x = arr.astype(np.float32) / 255.0

    x = exposure.rescale_intensity(x, in_range="image", out_range=(0,1))

    h, w = x.shape
    scale = max(h, w) / max_side
    if scale > 1.0:
        new_h, new_w = int(h/scale), int(w/scale)
        x = resize(x, (new_h, new_w), anti_aliasing=True, preserve_range=True).astype(np.float32)

    return x

# -----------------------
# Métricas (rápidas y estables)
# sigma_hat: proxy robusto con high-pass (evita estimate_sigma lento)
# -----------------------
def sigma_hat_fast(x):
    hp = x - gaussian(x, 1.0, preserve_range=True)
    med = np.median(hp)
    mad = np.median(np.abs(hp - med))
    return float(1.4826 * mad)

def rms_contrast(x): return float(np.std(x))
def sharpness_lapvar(x): return float(np.var(laplace(x)))
def edge_density(x, sigma=1.2): return float(canny(x, sigma=sigma).mean())

def histogram(x, bins=256):
    h, _ = np.histogram(x.ravel(), bins=bins, range=(0,1), density=True)
    return h

def hist_intersection(h1, h2): return float(np.minimum(h1, h2).sum())

def binarize_otsu(x):
    t = threshold_otsu(x)
    return (x >= t).astype(np.uint8)

def morph_stats(x):
    bw = binarize_otsu(x)
    lab = label(bw)
    props = regionprops_table(lab, properties=("area","perimeter"))
    if len(props["area"]) == 0:
        return {"n_comp": 0, "area_mean": 0.0, "area_sum": 0.0, "perim_mean": 0.0}
    areas = np.asarray(props["area"], dtype=float)
    perim = np.asarray(props["perimeter"], dtype=float)
    return {
        "n_comp": int(len(areas)),
        "area_mean": float(areas.mean()),
        "area_sum": float(areas.sum()),
        "perim_mean": float(perim.mean()),
    }

def compute_metrics(x0, x1):
    h0, h1 = histogram(x0), histogram(x1)
    return dict(
        ssim_vs_orig=float(ssim(x0, x1, data_range=1.0)),
        sigma_est=sigma_hat_fast(x1),
        rms_contrast=rms_contrast(x1),
        sharp_lapvar=sharpness_lapvar(x1),
        edge_density=edge_density(x1, sigma=1.2),
        entropy=float(shannon_entropy(x1)),
        hist_intersection=hist_intersection(h0, h1),
    )

# -----------------------
# Filtros
# -----------------------
def spatial_filters(x):
    return {
        "gaussian_s1": gaussian(x, 1.0, preserve_range=True),
        "gaussian_s2": gaussian(x, 2.0, preserve_range=True),
        "median_r2": median(x, footprint=disk(2)),
        "median_r4": median(x, footprint=disk(4)),
        "unsharp_r1_a1": exposure.rescale_intensity(x + (x - gaussian(x, 1.0))*1.0, out_range=(0,1)),
        "sobel": exposure.rescale_intensity(sobel(x), out_range=(0,1)),
        "laplace": exposure.rescale_intensity(np.abs(laplace(x)), out_range=(0,1)),
        "canny_s12": canny(x, 1.2).astype(np.float32),
    }

def morph_filters(x, r):
    se = disk(r)
    return {
        f"opening_r{r}": opening(x, se),
        f"closing_r{r}": closing(x, se),
        f"tophat_r{r}": white_tophat(x, se),
        f"blackhat_r{r}": black_tophat(x, se),
        f"erosion_r{r}": erosion(x, se),
        f"dilation_r{r}": dilation(x, se),
    }

def pipelines(x):
    den = median(x, footprint=disk(3))
    p1 = exposure.rescale_intensity(den + (den - gaussian(den, 1.5))*1.2, out_range=(0,1))
    p2 = opening(closing(x, disk(3)), disk(3))
    p3 = white_tophat(gaussian(x, 1.0, preserve_range=True), disk(3))
    return {
        "P1_median3_then_unsharp": p1,
        "P2_close3_then_open3": p2,
        "P3_gauss1_then_tophat3": p3,
    }

# -----------------------
# Ejecutar experimento
# -----------------------
records = []
baseline = {}

for p in img_paths:
    img_id = p.stem
    x = load_gray(p, max_side=512)
    baseline[img_id] = morph_stats(x)

    # espaciales + pipelines
    for name, y in spatial_filters(x).items():
        records.append({"image_id": img_id, "tipo": "espacial", "filtro": name, **compute_metrics(x, y), **morph_stats(y)})
    for name, y in pipelines(x).items():
        records.append({"image_id": img_id, "tipo": "pipeline", "filtro": name, **compute_metrics(x, y), **morph_stats(y)})

    # morfológicos (dos radios)
    for r in (2,4):
        for name, y in morph_filters(x, r).items():
            records.append({"image_id": img_id, "tipo": "morfologico", "filtro": name, **compute_metrics(x, y), **morph_stats(y)})

df = pd.DataFrame(records)
df.to_csv(RES_DIR / "metrics.csv", index=False)

# -----------------------
# Ranking top-3 (criterio combinado)
# -----------------------
def rank_score(sub):
    ssim_n = (sub["ssim_vs_orig"] - sub["ssim_vs_orig"].min()) / (sub["ssim_vs_orig"].max() - sub["ssim_vs_orig"].min() + 1e-9)
    sig_n  = (sub["sigma_est"] - sub["sigma_est"].min()) / (sub["sigma_est"].max() - sub["sigma_est"].min() + 1e-9)
    return ssim_n - 0.6*sig_n

top_list = []
for img_id, sub in df[df["tipo"].isin(["espacial","pipeline"])].groupby("image_id"):
    s = sub.copy()
    s["score"] = rank_score(s)
    top_list.append(s.sort_values("score", ascending=False).head(3))
top3 = pd.concat(top_list).sort_values(["image_id","score"], ascending=[True,False])

# -----------------------
# Selección morfológica top-2 (reduce componentes sin destrozar área)
# -----------------------
baseline_df = pd.DataFrame([{"image_id": k, **v} for k,v in baseline.items()]).set_index("image_id")
morph = df[df["tipo"]=="morfologico"].copy()

def morph_score(row):
    b = baseline_df.loc[row["image_id"]]
    area_rel = abs(row["area_sum"] - b["area_sum"]) / (b["area_sum"] + 1e-9)
    n_rel = row["n_comp"] / (b["n_comp"] + 1e-9)
    return -(0.7*area_rel + 0.3*n_rel)

morph["morph_score"] = morph.apply(morph_score, axis=1)
best2 = morph.sort_values("morph_score", ascending=False).groupby("image_id").head(2)

# -----------------------
# Figuras
# -----------------------
def save_grid_originals(paths, out):
    fig, axes = plt.subplots(3,3, figsize=(9,9))
    for ax, p in zip(axes.ravel(), paths):
        x = load_gray(p, max_side=512)
        ax.imshow(x, cmap="gray", vmin=0, vmax=1)
        ax.set_title(p.stem, fontsize=9)
        ax.axis("off")
    fig.tight_layout()
    fig.savefig(out, dpi=170)
    plt.close(fig)

def save_montage(img_path, out):
    x = load_gray(img_path, max_side=512)
    S = spatial_filters(x)
    M = morph_filters(x, 4)
    P = pipelines(x)

    picks = [
        ("original", x),
        ("gaussian_s1", S["gaussian_s1"]),
        ("median_r2", S["median_r2"]),
        ("unsharp_r1_a1", S["unsharp_r1_a1"]),
        ("sobel", S["sobel"]),
        ("closing_r4", M["closing_r4"]),
        ("opening_r4", M["opening_r4"]),
        ("P1_med3+unsharp", P["P1_median3_then_unsharp"]),
    ]

    cols = 4
    rows = 2
    fig, axes = plt.subplots(rows, cols, figsize=(13, 6.4))
    axes = axes.ravel()
    for ax, (lab, im) in zip(axes, picks):
        ax.imshow(im, cmap="gray", vmin=0, vmax=1)
        ax.set_title(lab, fontsize=9)
        ax.axis("off")
    fig.tight_layout()
    fig.savefig(out, dpi=170)
    plt.close(fig)

def save_hist(img_path, out):
    x = load_gray(img_path, max_side=512)
    S = spatial_filters(x)
    def h(im):
        hh, _ = np.histogram(im.ravel(), bins=256, range=(0,1), density=True)
        return hh

    fig = plt.figure(figsize=(7.5, 4.8))
    plt.plot(h(x), label="orig")
    plt.plot(h(S["gaussian_s1"]), label="gaussian_s1")
    plt.plot(h(S["unsharp_r1_a1"]), label="unsharp_r1_a1")
    plt.legend(fontsize=8)
    plt.title(f"Histogramas normalizados ({img_path.stem})")
    plt.xlabel("Bin de intensidad (0-1)")
    plt.ylabel("Densidad")
    plt.tight_layout()
    fig.savefig(out, dpi=170)
    plt.close(fig)

save_grid_originals(img_paths, FIG_DIR / "originals_grid.png")

# Montajes representativos (ajusta si quieres)
for k in ["I01","I04","I06","I07"]:
    save_montage(IMG_DIR / f"{k}.png", FIG_DIR / f"{k}_montage.png")
    save_hist(IMG_DIR / f"{k}.png", FIG_DIR / f"{k}_hist.png")

# -----------------------
# Tablas LaTeX
# -----------------------
# dataset_table: usa 'meta' de la celda anterior
dataset_tex = meta[["image_id","domain","source","label_or_category"]].to_latex(
    index=False, escape=True,
    caption="Imágenes seleccionadas (ID, dominio, procedencia y clase/categoría).",
    label="tab:dataset",
    column_format="llll"
)
(TAB_DIR / "dataset_table.tex").write_text(dataset_tex, encoding="utf-8")

top3_small = top3[["image_id","tipo","filtro","score","ssim_vs_orig","sigma_est","rms_contrast","edge_density"]].copy()
top3_small.rename(columns={"image_id":"ID","tipo":"Tipo","filtro":"Filtro",
                           "ssim_vs_orig":"SSIM","sigma_est":"sigma_hat",
                           "rms_contrast":"ContrasteRMS","edge_density":"DensidadBordes"}, inplace=True)
for c in ["score","SSIM","sigma_hat","ContrasteRMS","DensidadBordes"]:
    top3_small[c] = top3_small[c].astype(float).round(4)

top_tex = top3_small.to_latex(
    index=False, escape=True,
    caption="Top-3 filtros/pipelines por imagen según criterio combinado (SSIM alto y sigma\\_hat bajo).",
    label="tab:topfilters",
    column_format="lllrrrr"
)
(TAB_DIR / "topfilters_table.tex").write_text(top_tex, encoding="utf-8")

morph_small = best2[["image_id","filtro","morph_score","n_comp","area_sum","ssim_vs_orig"]].copy()
morph_small.rename(columns={"image_id":"ID","filtro":"Operacion","morph_score":"ScoreMorph",
                            "n_comp":"Ncomp","area_sum":"AreaSum","ssim_vs_orig":"SSIM"}, inplace=True)
morph_small["ScoreMorph"] = morph_small["ScoreMorph"].astype(float).round(4)
morph_small["SSIM"] = morph_small["SSIM"].astype(float).round(4)
morph_small["AreaSum"] = morph_small["AreaSum"].astype(float).round(0).astype(int)

morph_tex = morph_small.to_latex(
    index=False, escape=True,
    caption="Operaciones morfológicas destacadas (Top-2) por imagen. ScoreMorph favorece menor fragmentación con conservación del área tras Otsu.",
    label="tab:morph",
    column_format="llrrrr"
)
(TAB_DIR / "morph_table.tex").write_text(morph_tex, encoding="utf-8")

top3_small.head(), morph_small.head()


(     ID      Tipo       Filtro   score    SSIM  sigma_hat  ContrasteRMS  \
 0   I01  espacial  gaussian_s1  0.9018  0.9625     0.0038        0.1950   
 1   I01  espacial  gaussian_s2  0.8819  0.8810     0.0020        0.1920   
 2   I01  espacial    median_r2  0.8709  0.9660     0.0048        0.1958   
 24  I02  espacial  gaussian_s2  0.8352  0.8733     0.0022        0.2478   
 23  I02  espacial  gaussian_s1  0.8090  0.9593     0.0041        0.2524   
 
     DensidadBordes  
 0           0.0729  
 1           0.0497  
 2           0.0775  
 24          0.0587  
 23          0.0851  ,
       ID    Operacion  ScoreMorph  Ncomp  AreaSum    SSIM
 132  I06   opening_r4     -0.0164      1   100368  0.9080
 17   I01   opening_r4     -0.0179      4   120663  0.8768
 40   I02   opening_r4     -0.0247      7   117097  0.8510
 63   I03   opening_r4     -0.0261      9   122350  0.8032
 22   I01  dilation_r4     -0.0284      7   120864  0.6551)

In [ ]:
!zip -r results.zip /content/results


  adding: content/results/ (stored 0%)
  adding: content/results/metrics.csv (deflated 56%)
  adding: content/results/figures/ (stored 0%)
  adding: content/results/figures/I01_montage.png (deflated 2%)
  adding: content/results/figures/I06_montage.png (deflated 2%)
  adding: content/results/figures/I07_hist.png (deflated 5%)
  adding: content/results/figures/I07_montage.png (deflated 21%)
  adding: content/results/figures/I04_hist.png (deflated 7%)
  adding: content/results/figures/I06_hist.png (deflated 14%)
  adding: content/results/figures/I01_hist.png (deflated 4%)
  adding: content/results/figures/originals_grid.png (deflated 3%)
  adding: content/results/figures/I04_montage.png (deflated 4%)
  adding: content/results/tables/ (stored 0%)
  adding: content/results/tables/dataset_table.tex (deflated 56%)
  adding: content/results/tables/topfilters_table.tex (deflated 69%)
  adding: content/results/tables/morph_table.tex (deflated 58%)


In [ ]:
from google.colab import files
files.download("results.zip")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>